In [85]:
import os.path
import torch

PATH_TO_PRETRAINED = '/home/vika/comparative-dialogue/comparative-dialogue/external_pretrained_models/'
MODEL_NAME = 'model2_tagger.hdf5'

def load(checkpoint_fn, gpu=-1):
    if not os.path.isfile(PATH_TO_PRETRAINED + checkpoint_fn):
        raise ValueError('Can''t find tagger in file "%s". Please, run the main script with non-empty \
                         "--save-best-path" param to create it.' % checkpoint_fn)
    tagger = torch.load(PATH_TO_PRETRAINED + checkpoint_fn)
    tagger.gpu = gpu

    tagger.word_seq_indexer.gpu = gpu # hotfix
    tagger.tag_seq_indexer.gpu = gpu # hotfix
    if hasattr(tagger, 'char_embeddings_layer'):# very hot hotfix
        tagger.char_embeddings_layer.char_seq_indexer.gpu = gpu # hotfix
    tagger.self_ensure_gpu()
    return tagger

In [86]:
!pwd

/home/vika/comparative-dialogue/comparative-dialogue


In [87]:
def create_sequence_from_sentence(str_sentences):
    return [str_sentence.lower().split() for str_sentence in str_sentences]

In [88]:
words = create_sequence_from_sentence(['what is better amazon or itunes for showing', 'what is better mouse or rat', 'what is easier to make bread o pizza'])

In [ ]:
['what is better amazon or itunes for showing', 'what is better mouse or rat', 'what is easier to make bread o pizza']

In [89]:
import sys
sys.path.append('/home/vika/NER_RNN/targer')
from src.factories.factory_tagger import TaggerFactory

model = TaggerFactory.load(PATH_TO_PRETRAINED + MODEL_NAME)
tags = model.predict_tags_from_words(words)



++ predicting, batch 1/1 (0.00%).

In [90]:
def get_objects(list_words, list_tags):
    obj_list = []
    for ind, elem in enumerate(list_tags):
        if elem == 'B-OTHOBJ' or elem == 'B-ASPOBJ':
            obj_list.append(list_words[ind])
    return obj_list

In [91]:
objects_list = []
for elem in list(zip(words, tags)):
    objects = get_objects(elem[0], elem[1])
    assert len(objects) >= 2, "We have %d objects to compare" %(len(objects))
    objects_list.append((objects[0], objects[1]))

In [92]:
from util import get_response
from generation.generation import diviner

for obj0, obj1 in objects_list:
    response = get_response(obj0, obj1, False)
    response_json = response.json()
    Merlin = diviner()
    Merlin.create_from_json(response_json)
    Merlin.generate_advice()

get url
In this context, amazon are preferable to itunes, as it is faster, greater, deeper, faster for me, easier to interface here. Itunes are worse for consumers, easier, easier to work with, easier to navigate, easier to use store, better for random browsing and searching, cleaner
get url
Rat is better than mouse, because it is greater, easier to work with, bigger, easier, differentiation, easier to study diseases in rats, larger, easier to isolate and culture. At the same time, mouse is quicker, faster, shower, easier to pick up, easier to trap, richer
get url
In this context, pizza are preferable to bread, as it is faster, lighter, easier to make, better for your health, bigger, simpler, greater, easier to handle. Bread are safer, quicker, easier to find and more expensive, easier for me to make, nicer
